In [1]:
#divergencias
import util
from backtesting import Backtest
import estrategias as est

lista = ['ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'LTCUSDT', 'ETCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'DOGEUSDT', 'DOTUSDT', 
         'SOLUSDT', 'AVAXUSDT', 'NEARUSDT', 'FILUSDT', 'MATICUSDT', 'OPUSDT', 'FETUSDT', 'AGIXUSDT', 'ARBUSDT',  
         'SLPUSDT', 'MEMEUSDT',  '1000SATSUSDT', 'PIXELUSDT']
#lista=['ADAUSDT']

suma = 0.0
trades = 0
timeframe = '1h'
limit= 1000
sma_length = 0
sma_macd_length = 0
###
razon_max = 0.0
sma_length_max = 0.0
sma_macd_length_max = 0.0
razon = 0.0
for sma_length in [50]: #range(10,201,10): #
    for sma_macd_length in [80]: #range(10,201,10): # 
        print(f"timeframe: {timeframe} - limit: {limit} - sma_length: {sma_length} - sma_macd_length: {sma_macd_length}")
        suma = 0.0
        trades = 0
        razon = 0.0
        for symbol in lista:
            try:                
                data=est.estrategia_divergencias (symbol,timeframe,limit,sma_length,sma_macd_length)
                ###
                bt = Backtest(data,util.backtesting_config,cash=1000000,exclusive_orders=True)
                output = bt.run()
                if len(lista)==1:
                    bt.plot(filename="graficos/"+symbol)
                print(f"Symbol: {symbol} - Return [%]: {round(output['Return [%]'],2)} - # Trades: {output['# Trades']} - Win Rate [%]: {output['Win Rate [%]']}")
                suma=suma+output['Return [%]']
                trades=trades+output['# Trades']
            except Exception as e:
                print(f"Error en {symbol}: {e}")
        razon = suma/trades
        if razon > razon_max:
            razon_max = razon
            sma_length_max = sma_length
            sma_macd_length_max = sma_macd_length
        print(f"Suma total: {suma} - Trades total: {trades}")
        print(f"razon_max: {razon_max} - sma_length_max: {sma_length_max} - sma_macd_length_max: {sma_macd_length_max}")
print(f"razon_max: {razon_max} - sma_length_max: {sma_length_max} - sma_macd_length_max: {sma_macd_length_max}")

timeframe: 1h - limit: 1000 - sma_length: 50 - sma_macd_length: 80
Symbol: ETHUSDT - Return [%]: 5.64 - # Trades: 5 - Win Rate [%]: 40.0
Symbol: BCHUSDT - Return [%]: 13.01 - # Trades: 3 - Win Rate [%]: 100.0
Symbol: XRPUSDT - Return [%]: 0.68 - # Trades: 2 - Win Rate [%]: 50.0
Symbol: LTCUSDT - Return [%]: 7.41 - # Trades: 3 - Win Rate [%]: 66.66666666666666
Symbol: ETCUSDT - Return [%]: 6.64 - # Trades: 3 - Win Rate [%]: 66.66666666666666
Symbol: LINKUSDT - Return [%]: 1.32 - # Trades: 1 - Win Rate [%]: 100.0
Symbol: ADAUSDT - Return [%]: 0.73 - # Trades: 3 - Win Rate [%]: 66.66666666666666
Symbol: BNBUSDT - Return [%]: 5.78 - # Trades: 2 - Win Rate [%]: 100.0
Symbol: DOGEUSDT - Return [%]: -0.73 - # Trades: 2 - Win Rate [%]: 50.0
Symbol: DOTUSDT - Return [%]: -0.42 - # Trades: 1 - Win Rate [%]: 0.0
Symbol: SOLUSDT - Return [%]: 0.09 - # Trades: 1 - Win Rate [%]: 100.0
Symbol: AVAXUSDT - Return [%]: 4.99 - # Trades: 2 - Win Rate [%]: 100.0
Symbol: NEARUSDT - Return [%]: 2.02 - # Trad